In [1]:
import win32com.client as client
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By
import datetime as dt
from datetime import datetime
from PIL import Image
import os
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import win32gui
import win32con
import pandas as pd
import warnings
import smartsheet
import traceback

# Função para clicar em elementos padrão
def click(campo):
    time.sleep(3)
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, campo)))
    element.click()

# Função para dar 2 cliques em elementos padrão
def dois_click(campo):
    time.sleep(3)
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, campo)))
    actions = ActionChains(driver)
    actions.double_click(element)
    actions.perform()

# Função para escrever em campos input
def escrever(campo, texto):
    time.sleep(3)
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, campo)))
        element.clear()
        element.send_keys(texto)
    except TimeoutException:
        print("Timeout: Elemento de entrada não encontrado. Ignorando e continuando.")

# Função que considera tempo de execução
def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timer = '{:02d}:{:02d}'.format(mins, secs)
        print(timer, end="\r")
        time.sleep(1)
        t -= 1

# Crie um data frame a partir dos dados da planilha do excel
def carrega_dados(sheet):
    col_names = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    data_frame = pd.DataFrame(rows, columns=col_names)
    return data_frame

def update_smartsheet(column_title, cell_value, row_index):
    # Definindo variáveis globais para reutilizar posteriormente
    global smart
    global sheet_id
    global max_rows
    
    # Testa se o índice da linha é menor que a quantidade de linhas
    if row_index < max_rows:
        try:
            # Assume valores da planilha a partir do id da planilha do Smartsheet 
            sheet = smart.Sheets.get_sheet(sheet_id)
            rows = sheet.rows
            row_id = rows[row_index].id
            column_id = [col.id for col in sheet.columns if col.title == column_title][0]
            cell = smartsheet.models.Cell()
            cell.column_id = column_id
            
            # Atualiza valor da célula conforme o título da coluna e o row_index
            cell.value = cell_value
            updated_row = smartsheet.models.Row()
            updated_row.id = row_id
            updated_row.cells.append(cell)
            response = smart.Sheets.update_rows(sheet_id, [updated_row])
            
            # Printa valores dependendo do sucesso na requisição da API
            if response.message == 'SUCCESS':
                print(f'Valor "{cell_value}" atualizado na coluna "{column_title}" com sucesso.')
            else:
                print('Falha ao atualizar o valor na coluna.')
        except Exception as e:
            print(f'Erro: {e}')
    else:
        print('Índice da linha fora do intervalo.')
    return row_index

def click_webelement(xpath, indice):
    # Defzinha para clicar nos elementos usando como base o texto que eles possuem
    lista_campos = driver.find_elements(By.XPATH, xpath)  # Gera uma lista com todos os elementos encontrados que possuem aquele parâmetro
    campo_a_ser_clicado = lista_campos[indice]  # Armazena o elemento da lista que queremos clicar :D

    try:
        # Aguardar um pouco para que você possa ver o resultado
        time.sleep(5)
        # Aguardar até que o elemento esteja presente
        WebDriverWait(driver, 50).until(EC.visibility_of(campo_a_ser_clicado))
        campo_a_ser_clicado.click()
    except TimeoutException:
        # Se o elemento não for encontrado dentro do tempo de espera, imprimir uma mensagem
        print("Timeout: Elemento não encontrado. Ignorando e continuando.")

# Token de acesso da API Smartsheet
access_token = 'qGPQ1qbfWCpw07VytHUObAjZj5h2cAvHOBUet'
# Conecta à API
smart = smartsheet.Smartsheet(access_token)
# ID da planilha Smartsheet
sheet_id = '357594556223364'

# Assume valores da Planilha 
sheet = smart.Sheets.get_sheet(sheet_id)
# Carrega para o dataframe df
df = carrega_dados(sheet)

# Definindo linha de início e fim de execução 
row_index = 0
max_rows = len(df)

# Configurações do navegador webdriver
options = webdriver.EdgeOptions()
options.add_argument("--headless")
driver = webdriver.Edge()
driver.maximize_window()

# Define o tempo de início
start_time = time.time()

# Define a duração do loop em segundos (12 horas)
duration = 660 * 60

# Loop que roda por 19 minutos
while time.time() - start_time < duration:
    
    print("--------------INICIOU O PROCESSO!--------------")

    try:
        nome = df['CPF'].iloc[row_index]
        print(nome)
        link_whats = df['link_whats'].iloc[row_index]
        Resposta_Sim = df['Resposta_Sim'].iloc[row_index]
        Resposta_Nao = df['Resposta_Nao'].iloc[row_index]
        CR = df['CR'].iloc[row_index]
        print(Resposta_Nao, Resposta_Sim)

        if Resposta_Sim == None or Resposta_Nao == None or CR == "Enviado":
            driver.get(link_whats)
            time.sleep(10)
            click('/html/body/div[1]/div[1]/div[2]/div/section/div/div/div/div[2]/div[1]/a/span')
            click('/html/body/div[1]/div[1]/div[2]/div/section/div/div/div/div[3]/div/div/h4[2]/a/span')
            time.sleep(10)
            click('//*[@id="main"]/header/div[3]/div/div[2]/div')
            click('//*[@id="app"]/div/div[2]/div[5]/span/div/div/div[1]/div/div[2]/div[2]/div/div/p')
            escrever('//*[@id="app"]/div/div[2]/div[5]/span/div/div/div[1]/div/div[2]/div[2]/div/div/p', "⬇️⬇️Responda aqui.⬇️⬇️")
            click('/html/body/div[1]/div/div/div[2]/div[5]/span/div/div/div[2]/div[1]/div/div/div/div/div/div[2]/div[2]')
            
            click_webelement('//*[@title="Mostrar votos"]', 0)

            # Recarregar o DataFrame a cada iteração
            sheet = smart.Sheets.get_sheet(sheet_id)
            df = carrega_dados(sheet)
            max_rows = len(df)

            # Testa se a index da linha é menor que a quantidade de linhas
            if row_index >= max_rows:
                row_index = 0

            # Definindo variáveis conforme o row_index
            resp_1 = '/html/body/div[1]/div/div/div[2]/div[5]/span/div/span/div/div/div[2]/div[1]'          
            elemento_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, resp_1)))

            resposta_txt_1 = elemento_link.get_attribute('innerText')
            # Remover quebras de linha
            string_unica_linha_1 = resposta_txt_1.replace("\n", " ")

            print(string_unica_linha_1)

            resp_2 = '//*[@id="app"]/div/div[2]/div[5]/span/div/span/div/div/div[3]/div'          
            elemento_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, resp_2)))

            resposta_txt_2 = elemento_link.get_attribute('innerText')
            string_unica_linha_2 = resposta_txt_2.replace("\n", " ")

            print(string_unica_linha_2)

            if string_unica_linha_1 == "NÃO 1 voto":
                print(1)
                update_smartsheet("Resposta_Nao", string_unica_linha_1, row_index)
                update_smartsheet("Resposta_Sim", string_unica_linha_2, row_index)
            else:
                print(2)
                update_smartsheet("Resposta_Nao", string_unica_linha_2, row_index)
                update_smartsheet("Resposta_Sim", string_unica_linha_1, row_index)
        
        row_index += 1
        
    except Exception as e:
        print(f'Erro na iteração {row_index}: {e}')
        row_index += 1
        
    
    # Aguarda 5 segundos antes de iniciar o próximo loop
    time.sleep(5)


--------------INICIOU O PROCESSO!--------------
62092014900
NÃO 0 voto SIM 1 voto
Erro na iteração 412: Message: element not interactable
  (Session info: MicrosoftEdge=127.0.2651.105)
Stacktrace:
	GetHandleVerifier [0x00007FF7A99F05A2+11762]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF7A99908E9+271401]
	(No symbol) [0x00007FF7A97AFCD5]
	(No symbol) [0x00007FF7A97F7C23]
	(No symbol) [0x00007FF7A97ECA4C]
	(No symbol) [0x00007FF7A9815F3A]
	(No symbol) [0x00007FF7A97EC2FD]
	(No symbol) [0x00007FF7A97EC1CD]
	(No symbol) [0x00007FF7A9816220]
	(No symbol) [0x00007FF7A97EC2FD]
	(No symbol) [0x00007FF7A982FA97]
	(No symbol) [0x00007FF7A9815BA3]
	(No symbol) [0x00007FF7A97EB80C]
	(No symbol) [0x00007FF7A97EACED]
	(No symbol) [0x00007FF7A97EB3D1]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF7A9B6B524+879700]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF7A9925D4F+56159]
	Microsoft::Applications::Events::EventProperty::~